In [ ]:
%%capture output
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver




In [ ]:
%%shell
# Install selenium
pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Eudops_Project

/content/drive/MyDrive/Eudops_Project


This is the code for getting all the reviews from Saladlab, Crema

---



In [ ]:
import random
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

my_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument(f"--user-agent={my_user_agent}")

service = webdriver.chrome.service.Service("/usr/bin/chromedriver")

driver = webdriver.Chrome(service=service, options=options)

# IF the item in the page does not exists, return N/A
def extract_text_or_na(element):
    try:
        return element.get_text(strip=True)
    except AttributeError:
        return 'N/A'

# Scrape reviews from the page. Can handle review site by saladlab, from boardAlphareview
def get_reviews_from_page_alpha(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    data = soup.select('div.widget_item.review')

    if not data:
        return False

    half_point = len(data) // 2
    data = data[half_point:]

    review_data = []
    for item in data:
        review_number = item['value']
        review_product = extract_text_or_na(item.select_one('div.widget_product_name'))
        product_price = extract_text_or_na(item.select_one('div.widget_product_price'))
        product_type = extract_text_or_na(item.select_one('div.widget_item_tab_1_1'))
        username_1 = extract_text_or_na(item.select_one('span.widget_item_username_1'))
        username_2 = extract_text_or_na(item.select_one('span.widget_item_username_2'))
        review_rating = extract_text_or_na(item.select_one('div.widget_rating_number'))
        review_content = extract_text_or_na(item.select_one('span.widget_item_review_box'))
        review_timestamp = extract_text_or_na(item.select_one('div.widget_item_date'))

        review_data.append({
            'review_uid': review_number,
            'product_name': review_product,
            'product_price': product_price,
            'product_type': product_type,
            'username_1': username_1,
            'username_2': username_2,
            'rating': review_rating,
            'review_content': review_content,
            'review_date': review_timestamp
        })

    return review_data

def get_reviews_from_page_cafe24(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    data = soup.select('div.widget_m')

    if len(data) < 2:
        return False

    # For this api structure, the first widget_m contains review policy and will be disregarded
    data = data[1:]

    review_data = []
    for item in data:
        review_number = extract_text_or_na(item.select_one('div.widget_table_col_1'))
        review_product = extract_text_or_na(item.select_one('div.widget_product_name')) #Does not exits in this api
        product_price = extract_text_or_na(item.select_one('div.widget_product_price')) #Does not exits in this api
        product_type = extract_text_or_na(item.select_one('div.widget_item_tab_1_1')) #Does not exits in this api
        username_1 = extract_text_or_na(item.select_one('span.widget_item_username_1')) #Does not exits in this api
        username_2 = extract_text_or_na(item.select_one('div.widget_item_username'))

        review_rating = 5 - len(item.find_all("path", fill='#e6e9ec')) #Count the yellow stars
        review_content = extract_text_or_na(item.select_one('div.widget_table_col_2_expand_review_text'))
        review_timestamp = extract_text_or_na(item.select_one('div.widget_table_col_4'))

        review_data.append({
            'review_uid': review_number,
            'product_name': review_product,
            'product_price': product_price,
            'product_type': product_type,
            'username_1': username_1,
            'username_2': username_2,
            'rating': review_rating,
            'review_content': review_content,
            'review_date': review_timestamp
        })

    return review_data

def get_reviews_from_page_crema(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    data = soup.select('li.review_list_v2')

    if len(data) < 1:
        return False


    review_data = []
    for item in data:
        review_number = item['id'].split("_")[-1]
        review_product = extract_text_or_na(item.select_one('div.widget_product_name')) #Does not exits in this api
        product_price = extract_text_or_na(item.select_one('div.widget_product_price')) #Does not exits in this api
        product_type = extract_text_or_na(item.select_one('div.review_options_v2')) # Options for review - can be the type of user
        username_1 = "N/A" #Does not exits in this api
        username_2 = extract_text_or_na(item.select_one('div.review_list_v2__user_name_message').select_one('b'))

        review_rating = extract_text_or_na(item.select_one('span.visually-hidden'))[-2] # 별점: x점

        review_content = "N/A"
        if item.select_one('div.review_list_v2__content.review_list_v2__content--expanded'):
          review_content = extract_text_or_na(item.select_one('div.review_list_v2__content.review_list_v2__content--expanded').select_one('div.review_list_v2__message.js-translate-text'))
        elif item.select_one('div.review_list_v2__content.review_content__expanded'):
          review_content = extract_text_or_na(item.select_one('div.review_list_v2__content.review_content__expanded').select_one('div.review_list_v2__message.js-translate-text'))

        review_timestamp = extract_text_or_na(item.select_one('div.review_list_v2__date'))

        review_data.append({
            'review_uid': review_number,
            'product_name': review_product,
            'product_price': product_price,
            'product_type': product_type,
            'username_1': username_1,
            'username_2': username_2,
            'rating': review_rating,
            'review_content': review_content,
            'review_date': review_timestamp
        })

    return review_data

def get_reviews_from_page_smartstore(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    data = soup.select('li.review_list_v2')

    if len(data) < 1:
        return False


    review_data = []
    for item in data:
        review_number = item['id'].split("_")[-1]
        review_product = extract_text_or_na(item.select_one('div.widget_product_name')) #Does not exits in this api
        product_price = extract_text_or_na(item.select_one('div.widget_product_price')) #Does not exits in this api
        product_type = extract_text_or_na(item.select_one('div.review_options_v2')) # Options for review - can be the type of user
        username_1 = "N/A" #Does not exits in this api
        username_2 = extract_text_or_na(item.select_one('div.review_list_v2__user_name_message').select_one('b'))

        review_rating = extract_text_or_na(item.select_one('span.visually-hidden'))[-2] # 별점: x점

        review_content = "N/A"
        if item.select_one('div.review_list_v2__content.review_list_v2__content--expanded'):
          review_content = extract_text_or_na(item.select_one('div.review_list_v2__content.review_list_v2__content--expanded').select_one('div.review_list_v2__message.js-translate-text'))
        elif item.select_one('div.review_list_v2__content.review_content__expanded'):
          review_content = extract_text_or_na(item.select_one('div.review_list_v2__content.review_content__expanded').select_one('div.review_list_v2__message.js-translate-text'))

        review_timestamp = extract_text_or_na(item.select_one('div.review_list_v2__date'))

        review_data.append({
            'review_uid': review_number,
            'product_name': review_product,
            'product_price': product_price,
            'product_type': product_type,
            'username_1': username_1,
            'username_2': username_2,
            'rating': review_rating,
            'review_content': review_content,
            'review_date': review_timestamp
        })

    return review_data



Got page: 14829 ....appending now
Got page: 14830 ....appending now
Batch written to CSV
Got page: 14831 ....appending now
Got page: 14832 ....appending now
Got page: 14833 ....appending now
Got page: 14834 ....appending now
Got page: 14835 ....appending now
Got page: 14836 ....appending now
Got page: 14837 ....appending now
Got page: 14838 ....appending now
Got page: 14839 ....appending now
Got page: 14840 ....appending now
Batch written to CSV
Got page: 14841 ....appending now
Got page: 14842 ....appending now
Got page: 14843 ....appending now
Got page: 14844 ....appending now
Got page: 14845 ....appending now
Got page: 14846 ....appending now
Got page: 14847 ....appending now
Got page: 14848 ....appending now
Got page: 14849 ....appending now
Got page: 14850 ....appending now
Batch written to CSV
Got page: 14851 ....appending now
Got page: 14852 ....appending now
Got page: 14853 ....appending now
Got page: 14854 ....appending now
Got page: 14855 ....appending now
Got page: 14856 ...

In [ ]:
batch_data = []
file_dest = '/content/drive/My Drive/Eudops_Project/Product_Review_COKAN.csv'

# Load the website
for index in range(0, 0): #Used for Page Numbers

    #url = "https://review6.cre.ma/comfortlab.co.kr/products/reviews?app=0&iframe=1&iframe_id=crema-product-reviews-1&nonmember_token=&page=" + str(index) + "&parent_url=https%3A%2F%2Fwww.comfortlab.co.kr%2Fshop%2Fshopdetail.html%3Fbranduid%3D3355211%26xcode%3D061%26mcode%3D002%26scode%3D001%26type%3DY%26sort%3Dsellcnt%26cur_code%3D061%26search%3D%26GfDT%3DamZ3UQ%253D%253D&product_code=3355211&secure_device_token=V2ed57676649210a1896527c27ad08bbe6640004b8c9ec096dce2bbf0d632f7113b6f99df5024c2110132530ea2e4e11c2&widget_env=100&widget_style="

    driver.get(url)
    time.sleep(random.randint(1, 2))
    print("Got page: " + str(index) + " ....appending now")

    if "/boardAlphareview/" in url:
      page_data = get_reviews_from_page_alpha(driver.page_source)

    if "/boardCafe24" in url:
      page_data = get_reviews_from_page_cafe24(driver.page_source)

    if "cre.ma/" in url:
      page_data = get_reviews_from_page_crema(driver.page_source)

    if not page_data:
        break

    batch_data.extend(page_data)

    if index % 10 == 0:  # write in batches of 10
        df = pd.DataFrame(batch_data)
        df.to_csv(file_dest, mode='a', index=False, header=False)
        print("Batch written to CSV")
        batch_data = []

if batch_data:  # write the remaining data
    df = pd.DataFrame(batch_data)
    df.to_csv(file_dest, mode='a', index=False, header=False)

driver.quit()  # Close the browser after processing